In [1]:
!pip install torch

In [2]:
!pip install gradio


In [3]:
!pip install transformers

In [2]:
labels = ['일반적인 말', '부정적인 말', '반영', '지시', '행동 묘사', '질문', '일반적인 칭찬', '구체적인 칭찬']
label_dict = {}
for index, label in enumerate(labels):
    label_dict[label] = index
label_dict

{'일반적인 말': 0,
 '부정적인 말': 1,
 '반영': 2,
 '지시': 3,
 '행동 묘사': 4,
 '질문': 5,
 '일반적인 칭찬': 6,
 '구체적인 칭찬': 7}

In [3]:
label_dict
reverse_label_dict = dict(map(reversed,label_dict.items()))
reverse_label_dict

{0: '일반적인 말',
 1: '부정적인 말',
 2: '반영',
 3: '지시',
 4: '행동 묘사',
 5: '질문',
 6: '일반적인 칭찬',
 7: '구체적인 칭찬'}

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr

# 모델 및 토크나이저 로드
# model_name = "beomi/KcELECTRA-base-v2022"
model_name = "JuneKo/kcELECTRA_parent_09252023_MLM"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# model_save_path = "KcELECTRA_classification_epoch3_maxlength16.pth"
model_save_path = "KcELECTRA_DAPT_classification_epoch6_maxlength16.pth"
# 모델 아키텍처 생성
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_dict))

# 저장된 가중치 불러오기
loaded_model.load_state_dict(torch.load(model_save_path))

# 모델을 평가 모드로 설정
loaded_model.eval()

# input_encodings = tokenizer(input_data, padding=True, truncation=True, return_tensors="pt")

# # 모델에 입력 데이터 전달
# with torch.no_grad():
#     output = loaded_model(**input_encodings)

# # 예측 결과 확인
# logits = output.logits
# predicted_labels = logits.argmax(dim=1)

# 모델 추론 함수
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    outputs = loaded_model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return f"{reverse_label_dict[predicted_class]}"

# Gradio 웹 인터페이스 생성
iface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(label = "텍스트"),
    outputs=gr.Textbox(label = "DPICS 코드"),
    title="DPICS 자동 코딩 시스템"
)

if __name__ == "__main__":
    iface.launch(share=True)


Some weights of the model checkpoint at JuneKo/kcELECTRA_parent_09252023_MLM were not used when initializing ElectraForSequenceClassification: ['generator_lm_head.bias', 'generator_predictions.dense.weight', 'generator_predictions.dense.bias', 'generator_lm_head.weight', 'generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at JuneKo/kcELECTRA_parent_09252023_MLM and are new

Running on local URL:  http://127.0.0.1:7870
Running on public URL: https://fb27fc1c4656c3450c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
